.. _cartesian-axes:

# Cartesian Axes

In Toyplot, :class:`cartesian axes<toyplot.axes.Cartesian>` provide a traditional mapping of two-dimensional data values on the plane to canvas coordinates.  The axes *range* (the area on the canvas that they occupy) is specified when they are created (see :ref:`canvas-layout`).  Their *domain* is implicitly defined to include all of the data in the plot (but can be manually overridden by the caller if desired).

Cartesian axes are either created for you implicitly when using the :ref:`convenience-api`:

In [1]:
import numpy
y = numpy.linspace(0, 1, 20) ** 2

In [2]:
import toyplot
canvas, axes, mark = toyplot.plot(y, width=300)

... or explicitly using :meth:`toyplot.canvas.Canvas.axes`:

In [3]:
canvas = toyplot.Canvas(width=300)
axes = canvas.axes()
axes.plot(y);

## Properties

Axes objects contain sets of nested properties that can be used to adjust behavior.  The list of available properties includes the following:

* axes.show - set to *False* to hide the axes completely (the plotted data will still be visible).
* axes.aspect - set to "expand-domain" to expand the domain so that its aspect ratio matches the aspect ratio of the range.
* axes.padding - a small gap between the axes and their contents.  Defaults to CSS pixels, supports all :ref:`units`.
* axes.label.text - optional label at the top of the axes.
* axes.label.style - styles the axes label text.
* axes.coordinates.show - set to *False* to disable interactive mouse coordinates.
* axes.coordinates.style - styles the interactive mouse coordinates background.
* axes.coordinates.label.style - styles the interactive mouse coordinates text.
* axes.x.show - set to *False* to hide the X axis completely.
* axes.x.scale - "linear", "log" (base 10), "log10", "log2", or a ("log", base) tuple.
* axes.x.domain.min - override the minimum domain value for the axis.
* axes.x.domain.max - override the maximum domain value for the axis.
* axes.x.label.text - optional label below the X axis.
* axes.x.label.style - styles the X axis label.
* axes.x.spine.show - set to *False* to hide the X axis spine.
* axes.x.spine.position - set to "low", "high", or a Y axis domain value to position the spine.  Defaults to "low".
* axes.x.spine.style - styles the X axis spine.
* axes.x.ticks.show - set to *True* to display X axis tick marks.
* axes.x.ticks.locator - assign an instance of :py:class:`toyplot.locator.Basic`, :py:class:`toyplot.locator.Explicit`, :py:class:`toyplot.locator.Extended`, :py:class:`toyplot.locator.Heckbert`, or :py:class:`toyplot.locator.Log` to control the positioning and formatting of ticks and tick labels.  By default, an appropriate locator is automatically chosen based on the axis scale and domain.
* axes.x.ticks.above - length of X axis ticks above the axis.  Defaults to CSS pixels, supports all :ref:`units`.
* axes.x.ticks.below - length of X axis ticks below the axis.  Defaults to CSS pixels, supports all :ref:`units`.
* axes.x.ticks.style - styles the X axis ticks.
* axes.x.ticks.labels.show - set to *False* to hide X axis tick labels.
* axes.x.ticks.labels.angle - set the angle of X axis tick labels in degrees.
* axes.x.ticks.labels.offset - offsets labels from the axis.  Defaults to CSS pixels, supports all :ref:`units`.
* axes.x.ticks.labels.style - style X axis tick label text.
* ... and equivalent properties for the Y axis.

In the following example we override several of the defaults:

In [4]:
x = numpy.linspace(0, 2 * numpy.pi)
y = numpy.sin(x)

In [5]:
import toyplot.locator

canvas = toyplot.Canvas(width=600, height=300)
axes = canvas.axes()
axes.label.text = "Trigonometry 101"
axes.x.label.text = "x"
axes.y.label.text = "sin(x)"
axes.x.ticks.show = True
axes.x.ticks.locator = toyplot.locator.Explicit(
    [0, numpy.pi / 2, numpy.pi, 3 * numpy.pi / 2, 2 * numpy.pi],
    ["0", u"\u03c0 / 2", u"\u03c0", u"3 \u03c0 / 2", u"2 \u03c0"])
mark = axes.plot(x, y)

As a convenience, some of the most common properties can also be set when the axes are created:

In [6]:
x = numpy.linspace(0, 10, 100)
y = 40 + x ** 2

In [7]:
canvas = toyplot.Canvas(300, 300)
axes = canvas.axes(label="Toyplot Users", xlabel="Days", ylabel="Users")
mark = axes.plot(x, y)

And the same properties can be used with the :ref:`convenience-api`, as in the following example where we specify a minimum value for an axis - for example, if we wanted the previous figure to include $y = 0$:

In [8]:
toyplot.plot(x, y, label="Toyplot Users", xlabel="Days", ylabel="Users", ymin=0, width=300);

## Scale

An important property of each axis is its scale, used to specify linear or logarithmic mappings from *domain* to *range*:

In [9]:
x = numpy.linspace(-1000, 1000)

In [10]:
canvas = toyplot.Canvas(width=700)

axes = canvas.axes(grid=(2, 2, 0, 0), xscale="linear", yscale="linear")
axes.plot(x, x, marker="o")

axes = canvas.axes(grid=(2, 2, 0, 1), xscale="log", yscale="linear")
axes.plot(x, x, marker="o")

axes = canvas.axes(grid=(2, 2, 1, 0), xscale="linear", yscale="log")
axes.plot(x, x, marker="o")

axes = canvas.axes(grid=(2, 2, 1, 1), xscale="log", yscale="log")
axes.plot(x, x, marker="o");

Note that Toyplot handles negative values correctly, and provides sensible results for values near zero by rendering them using a small linear region around the origin.

The scale can be specified in two ways:

* As a string - "linear", "log" (base 10), "log10" (base 10), or "log2" (base 2).
* As a tuple - ("log", 2), ("log", 10).

For example, the following are all equivalent

In [11]:
canvas = toyplot.Canvas(width=700)
axes = canvas.axes(grid=(2,2,0), xscale="log")
axes.plot(x, x)
axes = canvas.axes(grid=(2,2,1), xscale="log10")
axes.plot(x, x)
axes = canvas.axes(grid=(2,2,2), xscale=("log", 10))
axes.plot(x, x);

Of course, you are free to specify any base you like, using the tuple notation:

In [12]:
toyplot.plot(x, x, xscale=("log", 4), width=400);

## Shared Axes

It may occasionally be useful to display multiple domains in a single plot, although this is a technique that should be used with care to avoid confusion when your plot is viewed by others.  For example, consider the following data:

In [13]:
import toyplot.data
data = toyplot.data.read_csv("deliveries.csv")
data[5:10]

Date,Delivered,On Time
15jul2005,306,1.0000
15aug2005,323,0.9905
15sep2005,531,0.9959
15oct2005,677,0.9600
15nov2005,695,0.9624


The `Delivered` and `On Time` series have completely different domains, and it would make little sense to plot counts and percentages on a single set of axes.  But with shared axes you can display the data using a separate Y axis for each series:

In [14]:
data["Delayed"] = 1.0 - data["On Time"].astype("float64")

canvas = toyplot.Canvas(width=600, height=300)
axes = canvas.axes(xlabel="Date", ylabel="Deliveries", ymin=0)
axes.plot(data["Delivered"], color="darkred", marker="o")
axes.y.spine.style = {"stroke":"darkred"}

axes = axes.share("x", ylabel="% Delayed", ymax=0.1)
axes.plot(data["Delayed"].astype("float64"), color="steelblue", marker="o")
axes.y.spine.style = {"stroke":"steelblue"}

For this example, we created a set of axes for the first series, then used the :func:`toyplot.axes.Cartesian.share` function to create a second set of overlapping axes for the second series.  Axis sharing in this case means that Toyplot has created two pairs of overlapping Cartesian axes, but with a shared X axis.  This ensures that any changes that affect the X axis are reflected in both plots.

Note that we have used color to help reinforce the relationship between the plots and their axes to avoid confusion.